<a href="https://colab.research.google.com/github/dnanper/TEST-langchain/blob/main/test04_agent_with_toolexecutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain-core>=0.3.47 langchain-openai>=0.3.9 langchain-community==0.3.16

In [2]:
!pip install langsmith

In [3]:
import os
from getpass import getpass

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "langchaintest"

Enter LangSmith API Key: ··········


In [4]:
import os
from getpass import getpass

os.environ["OPENROUTER_API_KEY"] = os.getenv("OPENROUTER_API_KEY") or getpass(
    "Enter OpenRouter API Key: "
)

openrouter_model = "deepseek/deepseek-r1:free"

Enter OpenRouter API Key: ··········


In [5]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    temperature=0.0,
    api_key = os.getenv("OPENROUTER_API_KEY"),
    base_url = "https://openrouter.ai/api/v1",
    model=openrouter_model
    )

In [6]:
from langchain_core.tools import tool

@tool
def add(x:float, y:float) -> float:
  """Add 'x' and 'y'."""
  return x+y

@tool
def multiply(x:float, y:float) -> float:
  """Multiply 'x' and 'y'."""
  return x*y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

In [7]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x7bba658d22a0>)

In [8]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

In [9]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict #ket qua cuoi cung ma LLM tra ve

{'x': 5, 'y': 2}

In [10]:
exponentiate.func(**llm_output_dict) # cho vao tool executor

25

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder # cho scratchpad

In [12]:
prompt = ChatPromptTemplate.from_messages([
    ("system",  (
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user."
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [13]:
from langchain_core.runnables.base import RunnableSerializable

In [14]:
tools = [add, multiply, exponentiate, subtract]

In [15]:
agent: RunnableSerializable = (
  {
      "input": lambda x : x["input"],
      "chat_history": lambda x : x["chat_history"],
      "agent_scratchpad": lambda x : x.get("agent_scratchpad", [])
  }
  | prompt
  | llm.bind_tools(tools, tool_choice="any") # must choose tool
)

In [26]:
# tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []}) -> không thực hiện được do openrouter api không cho phép tool use
# tool_call
from langchain_core.messages import AIMessage
tool_call = AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SPRcOwtqvcSRQdLaIx1U3ZBN', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 197, 'total_tokens': 215, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-47a97ee0-09e7-449b-8312-7e4cf524ae7d-0', tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': 'call_SPRcOwtqvcSRQdLaIx1U3ZBN', 'type': 'tool_call'}], usage_metadata={'input_tokens': 197, 'output_tokens': 18, 'total_tokens': 215, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})
"""
AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SPRcOwtqvcSRQdLaIx1U3ZBN', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}],
'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 197, 'total_tokens': 215, 'completion_tokens_details':
{'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-47a97ee0-09e7-449b-8312-7e4cf524ae7d-0',
tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': 'call_SPRcOwtqvcSRQdLaIx1U3ZBN', 'type': 'tool_call'}], usage_metadata={'input_tokens': 197, 'output_tokens': 18,
'total_tokens': 215, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})
"""

'\nAIMessage(content=\'\', additional_kwargs={\'tool_calls\': [{\'id\': \'call_SPRcOwtqvcSRQdLaIx1U3ZBN\', \'function\': {\'arguments\': \'{"x":10,"y":10}\', \'name\': \'add\'}, \'type\': \'function\'}], \n\'refusal\': None}, response_metadata={\'token_usage\': {\'completion_tokens\': 18, \'prompt_tokens\': 197, \'total_tokens\': 215, \'completion_tokens_details\': \n{\'accepted_prediction_tokens\': 0, \'audio_tokens\': 0, \'reasoning_tokens\': 0, \'rejected_prediction_tokens\': 0}, \'prompt_tokens_details\': {\'audio_tokens\': 0, \'cached_tokens\': 0}}, \n\'model_name\': \'gpt-4o-mini-2024-07-18\', \'system_fingerprint\': \'fp_bd83329f63\', \'finish_reason\': \'tool_calls\', \'logprobs\': None}, id=\'run-47a97ee0-09e7-449b-8312-7e4cf524ae7d-0\', \ntool_calls=[{\'name\': \'add\', \'args\': {\'x\': 10, \'y\': 10}, \'id\': \'call_SPRcOwtqvcSRQdLaIx1U3ZBN\', \'type\': \'tool_call\'}], usage_metadata={\'input_tokens\': 197, \'output_tokens\': 18, \n\'total_tokens\': 215, \'input_token_deta

In [30]:
"""
vì ta đã đặt là "any" cho tool choice nên luôn bắt buộc llm response bằng tool. Do đó content = empty vì field này chỉ dành cho Natural Language
để tìm tool output, ta cần check tool_call:
"""
tool_call.tool_calls

[{'name': 'add',
  'args': {'x': 10, 'y': 10},
  'id': 'call_SPRcOwtqvcSRQdLaIx1U3ZBN',
  'type': 'tool_call'}]

In [31]:
"""
tuy nhiên, LLM không tích hợp sẵn executor, nên dù có implement của hàm và các args truyền vào cho nó, vẫn không thể thực hiện hàm
ta cần tự viết phần thực hiện hàm
Thực hiện tool cần 2 bước:
 - maping tên hàm mà LLM chọn (sinh ra) -> hàm đó
 - thực hiện hàm với các args được sinh
"""
# create tool name to function mapping
name2tool = {tool.name: tool.func for tool in tools}

In [32]:
# execute
tool_exec_content = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)
tool_exec_content

20

In [25]:
# outpt: 20 -> có nghĩa là đã thực hiện hàm

In [33]:
from langchain_core.messages import ToolMessage

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_exec_content}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)
tool_exec

ToolMessage(content='The add tool returned 20', tool_call_id='call_SPRcOwtqvcSRQdLaIx1U3ZBN')

In [38]:
# out = agent.invoke({
#     "input": "What is 10 + 10",
#     "chat_history": [],
#     "agent_scratchpad": [tool_call, tool_exec] # vì sao lại truyền vào scratchpad? ở đây, scratchpad dùng để lưu mọi hoạt động của LLM, ta coi như LLM của mình đã tích hợp tool
                                                 # executor, đã sinh ra tool_call và sau đó thực hiện những gì được gọi trong tool_call và trả về tool_exec
                                                 # do đó, srcratchpad sẽ gồm 2 thông tin đó
                                                 # tóm lại, ta đã mô phỏng việc execute tool ở ngoài, và truyền lại output vào llm
# })
"""
NotFoundError: Error code:
404 - {'error': {'message': 'No endpoints found that support tool use. To learn more about provider routing, visit: https://openrouter.ai/docs/provider-routing', 'code': 404}}
"""
out = AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mbzS6t3YlNoyPf6jihCVAhmc', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 227, 'total_tokens': 245, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bf39d855-dd41-4cdc-8247-5671cd4fdd6b-0', tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': 'call_mbzS6t3YlNoyPf6jihCVAhmc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 227, 'output_tokens': 18, 'total_tokens': 245, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [37]:
out

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mbzS6t3YlNoyPf6jihCVAhmc', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 227, 'total_tokens': 245, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bf39d855-dd41-4cdc-8247-5671cd4fdd6b-0', tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': 'call_mbzS6t3YlNoyPf6jihCVAhmc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 227, 'output_tokens': 18, 'total_tokens': 245, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [39]:
# dù đã có kết quả trong scratchpad, output vẫn không chứa "20" do llm bị bắt buộc phải sử dụng tool, và chưa có tool nào cho phép nó lấy final answer
# ta sẽ code tool đấy
@tool
def final_answer(answer: str, tools_used: list[str]) -> str:
    """
    Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the `scratchpad`.
    """
    return {"answer": answer, "tools_used": tools_used}

In [41]:
tools = [final_answer, add, subtract, multiply, exponentiate]
# we need to update our name2tool mapping too
name2tool = {tool.name: tool.func for tool in tools}

In [42]:
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")  # we're forcing tool use again
)

In [44]:
# tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
# tool_call.tool_calls # với thông tin hiện tại, llm sẽ vẫn chỉ sinh tool call chứa add, chưa có final_answer
# [{'name': 'add',
#   'args': {'x': 10, 'y': 10},
#   'id': 'call_gYc9NADUc97rn13XMiNXrF12',
#   'type': 'tool_call'}]

In [48]:
tool_out = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_out}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

# out = agent.invoke({
#     "input": "What is 10 + 10",
#     "chat_history": [],
#     "agent_scratchpad": [tool_call, tool_exec]
# })
# lần gọi thứ 2, agent có thông tin: tool_call đầu tiên (chứa add), tool_exec với kết quả là 20
# đồng thời, nó có tool final_answer trong tool set, nên sẽ sử dụng tool final_answer ở lần response này
out = AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LVqTyA6lpOpAhNIlq10TKqjg', 'function': {'arguments': '{"answer":"10 + 10 equals 20.","tools_used":["functions.add"]}', 'name': 'final_answer'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 299, 'total_tokens': 327, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5f37367d-83b6-46bb-87f4-4dd773b98caa-0', tool_calls=[{'name': 'final_answer', 'args': {'answer': '10 + 10 equals 20.', 'tools_used': ['functions.add']}, 'id': 'call_LVqTyA6lpOpAhNIlq10TKqjg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 299, 'output_tokens': 28, 'total_tokens': 327, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [49]:
out.tool_calls

[{'name': 'final_answer',
  'args': {'answer': '10 + 10 equals 20.', 'tools_used': ['functions.add']},
  'id': 'call_LVqTyA6lpOpAhNIlq10TKqjg',
  'type': 'tool_call'}]

In [52]:
"""
Because we see the final_answer tool here, we don't pass this back into our agent, and instead,
this tells us to stop execution and pass the args output onto our downstream process or user directly:
"""

"\nBecause we see the final_answer tool here, we don't pass this back into our agent, and instead, \nthis tells us to stop execution and pass the args output onto our downstream process or user directly:\n"

In [50]:
out.tool_calls[0]["args"]

{'answer': '10 + 10 equals 20.', 'tools_used': ['functions.add']}

Tóm lại, ta thấy để thêm executor vào luồng hoạt động của LLM, ta phải có bước thực hiện bên ngoài: lấy tool call (name, các args) rồi fetch vào tool executor và thực hiện
Sau đó, ta lại tiếp tục lấy output của tool executor để truyền lại vào scratchpad của llm
Như vậy, ta có thể code một LLM_with_executor Loop để tự động thực hiện các tác vụ trên

In [53]:
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage


class CustomAgentExecutor:
    chat_history: list[BaseMessage]

    def __init__(self, max_iterations: int = 3):
        self.chat_history = []
        self.max_iterations = max_iterations # số lần gọi tool tối đa cho một task
        self.agent: RunnableSerializable = (
            {
                "input": lambda x: x["input"],
                "chat_history": lambda x: x["chat_history"],
                "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
            }
            | prompt
            | llm.bind_tools(tools, tool_choice="any")  # we're forcing tool use again
        )

    def invoke(self, input: str) -> dict:
        # invoke the agent but we do this iteratively in a loop until
        # reaching a final answer
        count = 0
        agent_scratchpad = []

        ## vòng lặp chính: gọi (invoke) llm dựa trên scratchpad, lấy tool call từ output của nó, thực hiện bên ngoài,
        ## rồi fetch lại cho llm toolcall và toolexec (thông qua scracthpad)
        while count < self.max_iterations:
            # invoke a step for the agent to generate a tool call
            tool_call = self.agent.invoke({
                "input": input,
                "chat_history": self.chat_history,
                "agent_scratchpad": agent_scratchpad
            })
            # add initial tool call to scratchpad
            agent_scratchpad.append(tool_call)
            # otherwise we execute the tool and add it's output to the agent scratchpad
            tool_name = tool_call.tool_calls[0]["name"]
            tool_args = tool_call.tool_calls[0]["args"]
            tool_call_id = tool_call.tool_calls[0]["id"]
            tool_out = name2tool[tool_name](**tool_args)
            # add the tool output to the agent scratchpad
            tool_exec = ToolMessage(
                content=f"{tool_out}",
                tool_call_id=tool_call_id
            )
            agent_scratchpad.append(tool_exec)
            # add a print so we can see intermediate steps
            print(f"{count}: {tool_name}({tool_args})")
            count += 1
            # if the tool call is the final answer tool, we stop
            if tool_name == "final_answer":
                break
        # add the final output to the chat history
        final_answer = tool_out["answer"]
        self.chat_history.extend([
            HumanMessage(content=input),
            AIMessage(content=final_answer)
        ])
        # return the final answer in dict form
        return json.dumps(tool_out)

In [54]:
agent_executor = CustomAgentExecutor()

In [56]:
# agent_executor.invoke(input="What is 10 + 10")